# **Stock market news feed semantic analysis** *(Baseline LogReg)*

Ebben a notebookban az eddigi általam kibányászott, megszerzett adathalmazokat fogom a hagyományos bag of words és logistic regression módszerrel megvizsgálni. Ezek után n-gram modelleket is ki fogok próbálni. Az általa használt források és referenciák az eredményekhez:


*   https://colab.research.google.com/drive/1QPrBkh-KwX6qcUtiNWKp9rJoneBfGEVh#scrollTo=bQUJwMjYYN4- *(saját munka - átdolgozott)*
*   https://colab.research.google.com/drive/1MdpXGCj2fb3g1BI_XfF54OWLkYQCZBBy#scrollTo=LndWT2Kn-UMK *(saját baseline munka)*
*   https://www.kaggle.com/ndrewgele/omg-nlp-with-the-djia-and-reddit#Basic-Model-Training-and-Testing
*   https://www.kaggle.com/lseiyjg/use-news-to-predict-stock-markets





A használt adathalmazok alapján külön fejezeteket készítek és mindenhol jelzem a forrását és a megszerzésének a módját, ha saját bányászás eredménye.

## **A projekt előkészítése**

A Drive csatlakoztatása a szükséges fájlok későbbi betöltésére. A betöltés közvetlen a használat előtt fogom megtenni.

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


A szükséges könyvtárak betöltése a projekthez.

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import pandas_datareader as web
from numpy.random import MT19937
from numpy.random import RandomState, SeedSequence
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
from nltk.tokenize import word_tokenize  
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


A projektben használt makrók definiálása.

In [69]:
# Shuffle cycle number for the dataframe
SHUFFLE_CYCLE = 500

# Which dataset will be used
DATASET = 2

A reprodukálhatóság miatt definiálok egy seed-et a véletlen szám generátorhoz, amit a továbbiakban használni fogok.

In [70]:
# Random seed
RANDOM_SEED = 1234

# Numpy random seed
NP_SEED = 1234

# Max iteration for training
MAX_ITER = 100000

# Train size
TRAIN_SPLIT = 0.85

# Test size
TEST_SPLIT = 0.15

In [71]:
np.random.seed(NP_SEED)

## **KAG_REDDIT_WRLD_DJIA_DF (1)**

Ez az adathalmaz a top25 hírt tartalmazza a Reddit World News kategóriából 2008.08.08-2016.07.01 időtartamban. Ez nem általam gyűjtött adathalmaz, a forrása:
Sun, J. (2016, August). Daily News for Stock Market Prediction, Version 1. Retrieved 2021.02.19. from https://www.kaggle.com/aaron7sun/stocknews

Az adathalmaz betöltése a csatlakoztatott Drive-omból.

In [72]:
if DATASET == 1:
    # Copy the dataset to the local environment
    !cp "/content/drive/MyDrive/Combined_News_DJIA.csv" "Combined_News_DJIA.csv"

    # Check the copy is succesfull -> good if no assertation error
    read = !ls
    assert read[0].find("Combined_News_DJIA.csv") != -1

Az eredmények elmentésére és indexelésére az alábbi két tömböt fogom hasnzálni.

In [73]:
if DATASET == 1:
    model_type = ["Bag of words", "1,2 n-gram", "2,2 n-gram", 
                  "1,3 n-gram", "2,3 n-gram", "3,3 n-gram"]

    result = []              

Makró definiálás.

In [74]:
if DATASET == 1:
    # Number of merged news into one string
    ROWS = 2

### A szöveg előkészítése

Az adathalmaz betöltése.

In [75]:
if DATASET == 1:
    # Load the dataset 
    df_combined = pd.read_csv('Combined_News_DJIA.csv', index_col = "Date")

    # Show the dataframe
    print(df_combined.head())

Érdekességképpen a következőkben megvizsgálom, hogy az adathalmaz címkéi megfelelőek. A forrás szerint a címke 1, ha nőtt vagy azonos maradt az érték azon a napon, illetve 0, ha csökkent. (Adj Close adott napi értéke az előző napihoz viszonyítva)

In [76]:
if DATASET == 1:
    # Load the stock data
    df_stock = web.DataReader("DJIA", data_source="yahoo", start="2008-08-08", 
                              end="2016-07-01")
    
    # Show the stock data
    print(df_stock.head())

Az dátumok formátumát egységesre hozom az összehasonlítás érdekében.

In [77]:
if DATASET == 1:
    temp_day = []

    for day in range(len(df_stock)):
        temp_day.append(df_stock.index[day].date())

    df_stock.index = temp_day

    # Show the stock data
    print(df_stock.head())

Először a dátumok ellenőzöm, hogy megegyeznek-e.

In [78]:
if DATASET == 1:
    difference = []

    if len(df_combined) == len(df_stock):
        print("The lengths are the same!")

    for day in range(max(len(df_combined), len(df_stock))):
        if str(df_combined.index[day]) != str(df_stock.index[day]):
            print("There is difference at: " + str(day) + " index")
            print("News: " + str(df_combined.index[day]) + "\tStock: " + str(df_stock.index[day]))
            difference.append(day)

    if len(difference) is 0:
        print("The dates matched!")

A labelek ellenőrzése.

In [79]:
if DATASET == 1:
    difference = []

    for day in range(len(df_stock)):
        # label should be 1 -> rise
        if int(df_stock["Adj Close"][day]) >= int(df_stock["Adj Close"][day - 1]):
            if df_combined["Label"][day] != 1:
                difference.append(str(df_stock.index[day]))
                print("Problem at day " + str(df_stock.index[day]))
                print("Today: " + str(df_stock["Adj Close"][day]) +"\t\tYesterday: " + str(df_stock["Adj Close"][day - 1]) + "\t\tLabel: " + str(df_combined["Label"][day]) + "\n")

        # label should be 0 -> fall
        if int(df_stock["Adj Close"][day]) < int(df_stock["Adj Close"][day - 1]):
            if df_combined["Label"][day] != 0:
                difference.append(str(df_stock.index[day]))
                print("Problem at day " + str(df_stock.index[day]))
                print("Today: " + str(df_stock["Adj Close"][day]) +"\t\tYesterday: " + str(df_stock["Adj Close"][day - 1]) + "\t\tLabel: " + str(df_combined["Label"][day]) + "\n")

    print("All differences: " + str(len(difference)))      

Látható, hogy rossz a label pár helyen. Egy kis kutakodás után megtaláltam, hogy maga az árfolyam lekérdezésük volt hibás pár nap esetében, ezért ezeket javítom, majd elmentem a drive-omon a javítottat.

In [80]:
if DATASET == 1:
    # correct the wrong labels
    for row in difference:
        if df_combined.loc[row, "Label"] == 0:
            df_combined.loc[row, "Label"] = 1
        else:
            df_combined.loc[row, "Label"] = 0

    # check them
    for row in difference:
        print(str(row) + "\t\t" + str(df_combined.loc[row, "Label"]))

In [81]:
if DATASET == 1:
    # save to drive
    df_combined.to_csv('drive/MyDrive/Kaggle dataset/Reddit Top 25 DJIA/KAG_REDDIT_WRLD_DJIA_DF_corrected.csv')

    # Show the dataset
    print(df_combined.head())

A következőkben az esetleges adat nélküli napokat, illetve cellákat keresem meg és helyettesítem őket egy üres sztringgel. Ez a későbbi szövegfeldolgozás hibamentességéhez szükséges.

In [82]:
if DATASET == 1:
    # Find the cells with NaN and after the rows for them
    is_NaN = df_combined.isnull()
    row_has_NaN = is_NaN.any(axis = 1)
    rows_with_NaN = df_combined[row_has_NaN]

    # Replace them
    df_combined = df_combined.replace(np.nan, " ")

    # Check the process
    is_NaN = df_combined.isnull()
    row_has_NaN = is_NaN.any(axis = 1)
    rows_with_NaN = df_combined[row_has_NaN]

    assert len(rows_with_NaN) is 0

Ezek után az egy naphoz tartozó híreket közös sztringekbe fűzöm. Az egy sztringbe tartozó hírek számát makróval definiálom:


*   ROWS - egymásba fűzött hírek száma

Itt megtalálható már az első előkészítő algoritmusom, méghozzá a sztringek elején található b karakter eltávolítása.

In [83]:
if DATASET == 1:
    # Get column names
    combined_column_names = []
    for column in df_combined.columns:
      combined_column_names.append(column)

    # 2D array creation for the news based on macros
    COLUMNS = len(df_combined)
    news_sum = [[0 for i in range(COLUMNS)] for j in range(int((len(combined_column_names) - 1) / ROWS))]  

    # Show the column names
    print("Column names of the dataset:") 
    print(combined_column_names)

    # Merge the news
    for row in range(len(df_combined)):
      for column in range(int((len(combined_column_names) - 1) / ROWS)):
        temp = ""
        news = ""
        for word in range(ROWS):
          news = df_combined[combined_column_names[(column * ROWS) + (word + 1)]][row]
          # Remove the b character at the begining of the string
          if news[0] is "b":
            news = " " + news[1:]
          temp = temp + news
        news_sum[column][row] = temp

    # Show the first day second package of the news
    print("\nThe first day second package of the news:")
    print(news_sum[1][0])

Ezek után a korábbi oszlopokat(Top1, Top2...) kicserélem a csoportosításnak megfelelő számú oszlopokra és nevekre (News_1, News_2...), majd feltöltöm őket az összevont hírcsomagokkal.

In [84]:
if DATASET == 1:
    # Drop the old columns
    for column in range(len(combined_column_names) - 1):
      df_combined.drop(combined_column_names[column + 1], axis = 1, inplace = True)

    # Create the new columns with the merged news
    for column in range(int((len(combined_column_names) - 1) / ROWS)):
      colum_name = "News_" + str(column + 1)
      df_combined[colum_name] = news_sum[column]

    # Show the DataFrame
    print(df_combined.head())

Egy új dataframebe újracsoportosítom a hír blokkokat a címkéjükkel, már a dátumok nélkül.

In [85]:
if DATASET == 1:
    # The label column 
    LABEL_COLUMN = 0

    news_sum = []
    label_sum = []

    # Get the column names
    combined_column_names = []
    for column in df_combined.columns:
      combined_column_names.append(column)

    # Write out the column names 
    print(combined_column_names)
    print("\n")

    # Connect the merged news with the labels
    for column in range(len(df_combined)):
      for row in range(len(combined_column_names) - 1):
        news_sum.append(df_combined[combined_column_names[row + 1]][column])
        label_sum.append(df_combined[combined_column_names[LABEL_COLUMN]][column])

    # Create the new DataFrame
    df_sum_news_labels = pd.DataFrame(data = label_sum, index = None, columns = ["Label"])
    df_sum_news_labels["News"] = news_sum

    # Show it
    print(df_sum_news_labels.head())

Először a szövegek előfeldolgozásával kezdem: írásjelek eltávolítása, számok eltávolítása, felesleges szóközök eltávolítása, aztán minden szót kis kezdőbetűjü szóvá konvertálom.

In [86]:
if DATASET == 1:
    # Removing punctuations
    temp_news = []
    for line in news_sum:
      temp_attach = ""
      for word in line:
        temp = " "
        if word not in string.punctuation:
          temp = word
        temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    news_sum = temp_news
    temp_news = []

    # Remove numbers
    for line in news_sum:
      temp_attach = ""
      for word in line:
        temp = " "
        if not word.isdigit():
          temp = word
        temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    # Remove space
    for line in range(len(temp_news)):    
      temp_news[line] = " ".join(temp_news[line].split())

    # Converting headlines to lower case
    for line in range(len(temp_news)): 
        temp_news[line] = temp_news[line].lower()

    # Update the data frame
    df_sum_news_labels["News"] = temp_news

    # Show it
    print(df_sum_news_labels.head())

A következőkben az úgy nevezett töltelék szavakat (stop words) fogom eltávolítani.

In [87]:
if DATASET == 1:
    # Load the stop words
    stop_words = set(stopwords.words('english'))

    filtered_sentence = []
    news_sum = df_sum_news_labels["News"]

    # Remove stop words
    for line in news_sum:
      word_tokens = word_tokenize(line)
      temp_attach = ""
      for word in word_tokens:
        temp = " "
        if not word in stop_words:
          temp = temp + word
        temp_attach = temp_attach + "".join(temp)
      filtered_sentence.append(temp_attach)

    # Remove space
    for line in range(len(filtered_sentence)):    
      filtered_sentence[line] = " ".join(filtered_sentence[line].split())

    # Update the data frame
    df_sum_news_labels["News"] = filtered_sentence

    # Show the DataFrame
    print(df_sum_news_labels.head())

Az adathalmazban lévő nulla hosszú sztring csomagok megkeresése és a hozzájuk tartozó cellák törlése következik.

In [88]:
if DATASET == 1:
    news_sum = df_sum_news_labels["News"]
    null_indexes = []
    index = 0

    for line in news_sum:
      if line is "":
        null_indexes.append(index)
      index = index + 1

    print(null_indexes)

    for row in null_indexes:
      df_sum_news_labels = df_sum_news_labels.drop(row)

    news_sum = df_sum_news_labels["News"]
    null_indexes = []
    index = 0

    for line in news_sum:
      if line is "":
        null_indexes.append(index)
      index = index + 1
      
    assert len(null_indexes) is 0

Az adathalmaz véletlenszerű sorbarendezése.

In [89]:
if DATASET == 1:
    # Do the shuffle
    for i in range(SHUFFLE_CYCLE):
      df_sum_news_labels = shuffle(df_sum_news_labels, random_state = RANDOM_SEED)

    # Reset the index
    df_sum_news_labels.reset_index(inplace=True, drop=True)

    # Show the data frame
    print(df_sum_news_labels.head())

Az adathalmaz szétbontása tanító és validáló/tesztelő adathalmazokra, majd a szétbontás ellenőrzése mérettel és első elem kiíratásával.

In [90]:
if DATASET == 1:
    INPUT_SIZE = len(df_sum_news_labels)
    TRAIN_SIZE = int(TRAIN_SPLIT * INPUT_SIZE) 
    TEST_SIZE = int(TEST_SPLIT * INPUT_SIZE)

    # Split the dataset
    train = df_sum_news_labels[:TRAIN_SIZE] 
    test = df_sum_news_labels[TRAIN_SIZE:]

    # Print out the length
    print("Train data set length: " + str(len(train)))
    print("Test data set length: " + str(len(test)))
    print("Split summa: " + str(len(train) + len(test)))
    print("Dataset summa before split: " + str(len(df_sum_news_labels)))

    # check
    split_sum = len(train) + len(test)
    sum = len(df_sum_news_labels)
    assert split_sum == sum

In [91]:
if DATASET == 1:
    print(train.tail(1))

In [92]:
if DATASET == 1:
    print(test.head(1))

### Bag of words

Először a tanító adathalmaz híreit fűzöm össze egy tömbbe.

In [93]:
if DATASET == 1:
    train_headlines = []

    for row in range(0, len(train.index)):
        train_headlines.append(train.iloc[row, 1])

    # show the first
    print(train_headlines[0])

Ezek után vektorizálom őket.

In [94]:
if DATASET == 1:
    bow_vectorizer = CountVectorizer()
    bow_train = bow_vectorizer.fit_transform(train_headlines)
    print(bow_train.shape)

Egy logistic regression modellt fogok erre a tanító halmazra betanítani.

In [95]:
if DATASET == 1:
    bow_model = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    bow_model = bow_model.fit(bow_train, train["Label"])

A teszt adathalmaz előkészítése, majd becslés a modell segítségével a következő lépés.

In [96]:
if DATASET == 1:
    test_headlines = []

    for row in range(0,len(test.index)):
        test_headlines.append(test.iloc[row, 1])

    bow_test = bow_vectorizer.transform(test_headlines)
    bow_predictions = bow_model.predict(bow_test)

Az eredmények megjelenítése egy táblázatban.

In [97]:
if DATASET == 1:
    pd.crosstab(test["Label"], bow_predictions, rownames=["Actual"], colnames=["Predicted"])

A pontossága a modellnek.

In [98]:
if DATASET == 1:
    print (classification_report(test["Label"], bow_predictions))
    print (accuracy_score(test["Label"], bow_predictions))

    result.append(accuracy_score(test["Label"], bow_predictions))

A következőkben a top 10 legbefolyásolóbb sztringet jelenítem meg mind pozítiv és mind negatív irányba.

In [99]:
if DATASET == 1:
    bow_words = bow_vectorizer.get_feature_names()
    bow_coeffs = bow_model.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : bow_words, 
                            'Coefficient' : bow_coeffs})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [100]:
if DATASET == 1:
    print(coeffdf.tail(10))

### 2-gram modell

Hasonlóan az eddigiekhez vektorizálom a tanító adathalmazom, logistic regression modellt illesztek rá, becslést hajtok végre majd kiértékelem az eredményeket. Először a (1,2) n-gram modellel.

In [101]:
if DATASET == 1:
    gram_vectorizer_12 = CountVectorizer(ngram_range=(1,2))
    train_vectorizer_12 = gram_vectorizer_12.fit_transform(train_headlines)

    print(train_vectorizer_12.shape)

    gram_model_12 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    gram_model_12 = gram_model_12.fit(train_vectorizer_12, train["Label"])

    gram_test_12 = gram_vectorizer_12.transform(test_headlines)
    gram_predictions_12 = gram_model_12.predict(gram_test_12)

    print (classification_report(test["Label"], gram_predictions_12))
    print (accuracy_score(test["Label"], gram_predictions_12))

    result.append(accuracy_score(test["Label"], gram_predictions_12))

In [102]:
if DATASET == 1:
    gram_words_12 = gram_vectorizer_12.get_feature_names()
    gram_coeffs_12 = gram_model_12.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : gram_words_12, 
                            'Coefficient' : gram_coeffs_12})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [103]:
if DATASET == 1:
    print(coeffdf.tail(10))

Másodjára a (2,2) n-gram modellel.

In [104]:
if DATASET == 1:
    gram_vectorizer_22 = CountVectorizer(ngram_range=(2,2))
    train_vectorizer_22 = gram_vectorizer_22.fit_transform(train_headlines)

    print(train_vectorizer_22.shape)

    gram_model_22 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    gram_model_22 = gram_model_22.fit(train_vectorizer_22, train["Label"])

    gram_test_22 = gram_vectorizer_22.transform(test_headlines)
    gram_predictions_22 = gram_model_22.predict(gram_test_22)

    pd.crosstab(test["Label"], gram_predictions_22, rownames=["Actual"], colnames=["Predicted"])

    print (classification_report(test["Label"], gram_predictions_22))
    print (accuracy_score(test["Label"], gram_predictions_22))

    result.append(accuracy_score(test["Label"], gram_predictions_22))

In [105]:
if DATASET == 1:
    gram_words_22 = gram_vectorizer_22.get_feature_names()
    gram_coeffs_22 = gram_model_22.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : gram_words_22, 
                            'Coefficient' : gram_coeffs_22})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [106]:
if DATASET == 1:
    print(coeffdf.tail(10))

### 3-gram modell

Hasonlóan az eddigiekhez vektorizálom a tanító adathalmazom, logistic regression modellt illesztek rá, becslést hajtok végre majd kiértékelem az eredményeket. Először a (1,3) n-gram modellel.

In [107]:
if DATASET == 1:
    gram_vectorizer_13 = CountVectorizer(ngram_range=(1,3))
    train_vectorizer_13 = gram_vectorizer_13.fit_transform(train_headlines)

    print(train_vectorizer_13.shape)

    gram_model_13 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    gram_model_13 = gram_model_13.fit(train_vectorizer_13, train["Label"])

    gram_test_13 = gram_vectorizer_13.transform(test_headlines)
    gram_predictions_13 = gram_model_13.predict(gram_test_13)

    print (classification_report(test["Label"], gram_predictions_13))
    print (accuracy_score(test["Label"], gram_predictions_13))

    result.append(accuracy_score(test["Label"], gram_predictions_13))

In [108]:
if DATASET == 1:
    gram_words_13 = gram_vectorizer_13.get_feature_names()
    gram_coeffs_13 = gram_model_13.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : gram_words_13, 
                            'Coefficient' : gram_coeffs_13})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [109]:
if DATASET == 1:
    print(coeffdf.tail(10))

Hasonlóan az eddigiekhez vektorizálom a tanító adathalmazom, logistic regression modellt illesztek rá, becslést hajtok végre majd kiértékelem az eredményeket. Először a (2,3) n-gram modellel.

In [110]:
if DATASET == 1:
    gram_vectorizer_23 = CountVectorizer(ngram_range=(2,3))
    train_vectorizer_23 = gram_vectorizer_23.fit_transform(train_headlines)

    print(train_vectorizer_23.shape)

    gram_model_23 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    gram_model_23 = gram_model_23.fit(train_vectorizer_23, train["Label"])

    gram_test_23 = gram_vectorizer_23.transform(test_headlines)
    gram_predictions_23 = gram_model_23.predict(gram_test_23)

    pd.crosstab(test["Label"], gram_predictions_23, rownames=["Actual"], colnames=["Predicted"])

    print (classification_report(test["Label"], gram_predictions_23))
    print (accuracy_score(test["Label"], gram_predictions_23))

    result.append(accuracy_score(test["Label"], gram_predictions_23))

In [111]:
if DATASET == 1:
    gram_words_23 = gram_vectorizer_23.get_feature_names()
    gram_coeffs_23 = gram_model_23.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : gram_words_23, 
                            'Coefficient' : gram_coeffs_23})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [112]:
if DATASET == 1:
    print(coeffdf.tail(10))

Hasonlóan az eddigiekhez vektorizálom a tanító adathalmazom, logistic regression modellt illesztek rá, becslést hajtok végre majd kiértékelem az eredményeket. Először a (3,3) n-gram modellel.

In [113]:
if DATASET == 1:
    gram_vectorizer_33 = CountVectorizer(ngram_range=(3,3))
    train_vectorizer_33 = gram_vectorizer_33.fit_transform(train_headlines)

    print(train_vectorizer_33.shape)

    gram_model_33 = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    gram_model_33 = gram_model_33.fit(train_vectorizer_33, train["Label"])

    gram_test_33 = gram_vectorizer_33.transform(test_headlines)
    gram_predictions_33 = gram_model_33.predict(gram_test_33)

    pd.crosstab(test["Label"], gram_predictions_33, rownames=["Actual"], colnames=["Predicted"])

    print (classification_report(test["Label"], gram_predictions_33))
    print (accuracy_score(test["Label"], gram_predictions_33))

    result.append(accuracy_score(test["Label"], gram_predictions_33))

In [114]:
if DATASET == 1:
    gram_words_33 = gram_vectorizer_33.get_feature_names()
    gram_coeffs_33 = gram_model_33.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : gram_words_33, 
                            'Coefficient' : gram_coeffs_33})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
    print(coeffdf.head(10))

In [115]:
if DATASET == 1:
    print(coeffdf.tail(10))

### 4-gram modell

Ebben a fejezetben már egy ciklusban vizsgálom meg a bizonyos modelleket és mentem le az eredményeiket.

In [116]:
if DATASET == 1:
    for n in range(1,5):
        print("--------------------------------------------\n\nStart of the " 
              + str(n) + ",4 gram model\n")

        _gram_vectorizer_ = CountVectorizer(ngram_range=(n,4))
        _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

        print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

        _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
        _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

        _gram_test_ = _gram_vectorizer_.transform(test_headlines)
        _gram_predictions_ = _gram_model_.predict(_gram_test_)

        print (accuracy_score(test["Label"], _gram_predictions_))

        model_type.append(str(n) + ",4 n-gram")
        result.append(accuracy_score(test["Label"], _gram_predictions_))

### 5-gram modell

Ebben a fejezetben már egy ciklusban vizsgálom meg a bizonyos modelleket és mentem le az eredményeiket.

In [117]:
if DATASET == 1:
    MODEL_TYPE = 5

    for n in range(1,MODEL_TYPE+1):
        print("--------------------------------------------\n\nStart of the " 
              + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

        _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
        _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

        print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

        _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
        _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

        _gram_test_ = _gram_vectorizer_.transform(test_headlines)
        _gram_predictions_ = _gram_model_.predict(_gram_test_)

        print (accuracy_score(test["Label"], _gram_predictions_))

        model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
        result.append(accuracy_score(test["Label"], _gram_predictions_))

### 6-gram modell

Ebben a fejezetben már egy ciklusban vizsgálom meg a bizonyos modelleket és mentem le az eredményeiket.

In [118]:
if DATASET == 1:
    MODEL_TYPE = 6

    for n in range(1,MODEL_TYPE+1):
        print("--------------------------------------------\n\nStart of the " 
              + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

        _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
        _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

        print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

        _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
        _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

        _gram_test_ = _gram_vectorizer_.transform(test_headlines)
        _gram_predictions_ = _gram_model_.predict(_gram_test_)

        print (accuracy_score(test["Label"], _gram_predictions_))

        model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
        result.append(accuracy_score(test["Label"], _gram_predictions_))

### Eredmények összegzése

Az eredmények kiíratása, a legjobbat kiemelve.

In [119]:
if DATASET == 1:
    best_model = 0

    for model in range(len(model_type)):
        print(str(model_type[model]) + ":\t\t\t\t\t" + str(result[model]))

        if result[model] > best_model:
            best_model = result[model]
            best_model_index = model

    print("--------------------------------------------\nBest model:\n" 
          + str(model_type[best_model_index]) + "\t\t\t\t\t" + 
          str(result[best_model_index]))

### ROWS makró optimalizálás

Ebben a fejezetben a különböző ROWS értékekre (mennyi napi hírt fűzünk egybe) futtatom végig egy automatizált bag of words -> 6,6 gram modell tanítást és becslést és állapítom meg, hogy melyik a legpontosabb.

A tesztelendő paraméterek megadása.

In [120]:
if DATASET == 1:
    # Number of merged news into one string: 1...12, 25 
    rows_values = []
    for value in range(1,13):
        rows_values.append(value)

    rows_values.append(25)

    print(rows_values)

A modell típusok összegyűjtése az automatizált tanításhoz.

In [121]:
if DATASET == 1:
    model_type_values = []
    for value in range(1,7):
        model_type_values.append(value)

    print(model_type_values)

A paraméterhez tartozó eredmények tárolására létrehozom az alábbi tömböket.

In [122]:
if DATASET == 1:
    rows_summary_value = []
    rows_summary_accuraccy = []

Automatizált tanítás és mentések.

In [123]:
if DATASET == 1:
    def preprocess():
        df_combined = pd.read_csv('drive/MyDrive/Kaggle dataset/Reddit Top 25 DJIA/KAG_REDDIT_WRLD_DJIA_DF_corrected.csv', 
                                index_col = "Date")

        # Find the cells with NaN and after the rows for them
        is_NaN = df_combined.isnull()
        row_has_NaN = is_NaN.any(axis = 1)
        rows_with_NaN = df_combined[row_has_NaN]

        # Replace them
        df_combined = df_combined.replace(np.nan, " ")

        # Check the process
        is_NaN = df_combined.isnull()
        row_has_NaN = is_NaN.any(axis = 1)
        rows_with_NaN = df_combined[row_has_NaN]

        assert len(rows_with_NaN) is 0

        # Get column names
        combined_column_names = []
        for column in df_combined.columns:
          combined_column_names.append(column)

        # 2D array creation for the news based on macros
        COLUMNS = len(df_combined)
        news_sum = []
        news_sum = [[0 for i in range(COLUMNS)] for j in range(int((len(combined_column_names) - 1) / ROWS))]  

        # Merge the news
        for row in range(len(df_combined)):
          for column in range(int((len(combined_column_names) - 1) / ROWS)):
            temp = ""
            news = ""
            for word in range(ROWS):
              news = df_combined[combined_column_names[(column * ROWS) + (word + 1)]][row]
              # Remove the b character at the begining of the string
              if news[0] is "b":
                news = " " + news[1:]
              temp = temp + news
            news_sum[column][row] = temp

        # Drop the old columns
        for column in range(len(combined_column_names) - 1):
          df_combined.drop(combined_column_names[column + 1], axis = 1, inplace = True)

        # Create the new columns with the merged news
        for column in range(int((len(combined_column_names) - 1) / ROWS)):
          colum_name = "News_" + str(column + 1)
          df_combined[colum_name] = news_sum[column]          

        # The label column 
        LABEL_COLUMN = 0

        news_sum = []
        label_sum = []

        # Get the column names
        combined_column_names = []
        for column in df_combined.columns:
          combined_column_names.append(column)

        # Connect the merged news with the labels
        for column in range(len(df_combined)):
          for row in range(len(combined_column_names) - 1):
            news_sum.append(df_combined[combined_column_names[row + 1]][column])
            label_sum.append(df_combined[combined_column_names[LABEL_COLUMN]][column])

        # Create the new DataFrame
        df_sum_news_labels = pd.DataFrame(data = label_sum, index = None, columns = ["Label"])
        df_sum_news_labels["News"] = news_sum

        # Removing punctuations
        temp_news = []
        for line in news_sum:
          temp_attach = ""
          for word in line:
            temp = " "
            if word not in string.punctuation:
              temp = word
            temp_attach = temp_attach + "".join(temp)
          temp_news.append(temp_attach)

        news_sum = temp_news
        temp_news = []

        # Remove numbers
        for line in news_sum:
          temp_attach = ""
          for word in line:
            temp = " "
            if not word.isdigit():
              temp = word
            temp_attach = temp_attach + "".join(temp)
          temp_news.append(temp_attach)

        # Remove space
        for line in range(len(temp_news)):    
          temp_news[line] = " ".join(temp_news[line].split())

        # Converting headlines to lower case
        for line in range(len(temp_news)): 
            temp_news[line] = temp_news[line].lower()

        # Update the data frame
        df_sum_news_labels["News"] = temp_news

        # Load the stop words
        stop_words = set(stopwords.words('english'))

        filtered_sentence = []
        news_sum = df_sum_news_labels["News"]

        # Remove stop words
        for line in news_sum:
          word_tokens = word_tokenize(line)
          temp_attach = ""
          for word in word_tokens:
            temp = " "
            if not word in stop_words:
              temp = temp + word
            temp_attach = temp_attach + "".join(temp)
          filtered_sentence.append(temp_attach)

        # Remove space
        for line in range(len(filtered_sentence)):    
          filtered_sentence[line] = " ".join(filtered_sentence[line].split())

        # Update the data frame
        df_sum_news_labels["News"] = filtered_sentence

        news_sum = df_sum_news_labels["News"]
        null_indexes = []
        index = 0

        for line in news_sum:
          if line is "":
            null_indexes.append(index)
          index = index + 1

        for row in null_indexes:
          df_sum_news_labels = df_sum_news_labels.drop(row)

        news_sum = df_sum_news_labels["News"]
        null_indexes = []
        index = 0

        for line in news_sum:
          if line is "":
            null_indexes.append(index)
          index = index + 1
          
        assert len(null_indexes) is 0

        # Do the shuffle
        for i in range(SHUFFLE_CYCLE):
          df_sum_news_labels = shuffle(df_sum_news_labels, random_state = RANDOM_SEED)

        # Reset the index
        df_sum_news_labels.reset_index(inplace=True, drop=True)

        return df_sum_news_labels

In [124]:
if DATASET == 1:
    def split_to_train():
        INPUT_SIZE = len(df_sum_news_labels)
        TRAIN_SIZE = int(TRAIN_SPLIT * INPUT_SIZE) 

        # Split the dataset
        train = df_sum_news_labels[:TRAIN_SIZE] 

        return train

In [125]:
if DATASET == 1:
    def split_to_test():
        INPUT_SIZE = len(df_sum_news_labels)
        TRAIN_SIZE = int(TRAIN_SPLIT * INPUT_SIZE) 

        # Split the dataset
        test = df_sum_news_labels[TRAIN_SIZE:]

        return test

In [126]:
if DATASET == 1:
    for ROWS in rows_values:
      
        print("--------------------------------------------\n\nStart of the ROWS = " 
          + str(ROWS) + " sequence\n\n--------------------------------------------\n")
        
        model_type = []
        result = []

        df_sum_news_labels = preprocess()
        train = split_to_train()
        test = split_to_test()

        # check
        split_sum = len(train) + len(test)
        sum = len(df_sum_news_labels)
        assert split_sum == sum    

        train_headlines = []
        test_headlines = []

        for row in range(0, len(train.index)):
            train_headlines.append(train.iloc[row, 1])

        for row in range(0,len(test.index)):
            test_headlines.append(test.iloc[row, 1])

        # show the first
        print(train_headlines[0])

        for MODEL_TYPE in model_type_values:

            for n in range(1,MODEL_TYPE+1):
                print("--------------------------------------------\n\nStart of the " 
                      + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

                _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
                _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

                print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

                _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
                _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

                _gram_test_ = _gram_vectorizer_.transform(test_headlines)
                _gram_predictions_ = _gram_model_.predict(_gram_test_)

                print (accuracy_score(test["Label"], _gram_predictions_))

                model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
                result.append(accuracy_score(test["Label"], _gram_predictions_))

        rows_summary_value.append(ROWS)

        # save the best
        best_model_rows = 0

        for model in range(len(model_type)):
            if result[model] > best_model_rows:
                best_model_rows = result[model]

        rows_summary_accuraccy.append(best_model_rows)

Kiértékelés.

In [127]:
if DATASET == 1:
    best_model_rows = 0

    for model in range(len(rows_summary_value)):
        print(str(rows_summary_value[model]) + ":\t\t\t\t\t" 
              + str(rows_summary_accuraccy[model]))

        if rows_summary_accuraccy[model] > best_model_rows:
            best_model_rows = rows_summary_accuraccy[model]
            best_model_rows_index = model

    print("--------------------------------------------\nBest row value:\n" 
          + str(rows_summary_value[best_model_rows_index]) + "\t\t\t\t\t" + 
          str(rows_summary_accuraccy[best_model_rows_index]))

A legjobb ROWS eredményeinek megjelenítése.

In [128]:
if DATASET == 1:
    ROWS = int(rows_summary_value[best_model_rows_index])

    model_type = []
    result = []

    df_sum_news_labels = preprocess()
    train = split_to_train()
    test = split_to_test()

    # check
    split_sum = len(train) + len(test)
    sum = len(df_sum_news_labels)
    assert split_sum == sum    

    train_headlines = []
    test_headlines = []

    for row in range(0, len(train.index)):
        train_headlines.append(train.iloc[row, 1])

    for row in range(0,len(test.index)):
        test_headlines.append(test.iloc[row, 1])

    # show the first
    print(train_headlines[0])

    for MODEL_TYPE in model_type_values:

        for n in range(1,MODEL_TYPE+1):
            print("--------------------------------------------\n\nStart of the " 
                  + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

            _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
            _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

            print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

            _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
            _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

            _gram_test_ = _gram_vectorizer_.transform(test_headlines)
            _gram_predictions_ = _gram_model_.predict(_gram_test_)

            print (accuracy_score(test["Label"], _gram_predictions_))

            model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
            result.append(accuracy_score(test["Label"], _gram_predictions_))

    best_model_gram = 0

    for model in range(len(model_type)):
        print(str(model_type[model]) + ":\t\t\t\t\t" + str(result[model]))

        if result[model] > best_model_gram:
            best_model_gram = result[model]
            best_model_gram_index = model

    print("--------------------------------------------\nBest model:\n" 
          + str(model_type[best_model_gram_index]) + "\t\t\t\t\t" + 
          str(result[best_model_gram_index]))

A legjobbhoz tartozó korrelációs tényezők megjelenítése.

In [129]:
if DATASET == 1:
    ROWS = int(rows_summary_value[best_model_rows_index])
    MODEL_TYPE = str(model_type[best_model_gram_index])

    df_sum_news_labels = preprocess()
    train = split_to_train()
    test = split_to_test()

    # check
    split_sum = len(train) + len(test)
    sum = len(df_sum_news_labels)
    assert split_sum == sum    

    train_headlines = []
    test_headlines = []

    for row in range(0, len(train.index)):
        train_headlines.append(train.iloc[row, 1])

    for row in range(0,len(test.index)):
        test_headlines.append(test.iloc[row, 1])

    # show the first
    print(train_headlines[0])

    _gram_vectorizer_ = CountVectorizer(ngram_range=(int(MODEL_TYPE[0]),int(MODEL_TYPE[2])))
    _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

    print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

    _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["Label"])

    _gram_test_ = _gram_vectorizer_.transform(test_headlines)
    _gram_predictions_ = _gram_model_.predict(_gram_test_)

    print (accuracy_score(test["Label"], _gram_predictions_))

    model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
    result.append(accuracy_score(test["Label"], _gram_predictions_))

In [130]:
if DATASET == 1:
    _gram_words_best_ = _gram_vectorizer_.get_feature_names()
    _gram_coeffs_best_ = _gram_model_.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : _gram_words_best_, 
                            'Coefficient' : _gram_coeffs_best_})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])

    print(coeffdf.head(10))

In [131]:
if DATASET == 1:
    print(coeffdf.tail(10))

## **ECO_BSN_DF, ECO_FNC_DF, ECO_US_DF 2008-2016 (2)**

Megvizsgálom a reddit-es világhírekkel megegyező intervallumon ezeket az összevont adathalmazokat, majd egyesítve és kombinálva a kettőt megvizsgálom, hogy javítja-e a pontosságot.

Ezeket az adathalmazokat én magam gyűjtöttem az alábbi oldalakról:


*   https://www.economist.com/business/ 
*   https://www.economist.com/finance-and-economics/ 
*   https://www.economist.com/united-states/ 

### Adathalmazok betöltése

Először betöltöm külön-külön az adathalmazokat.

In [132]:
if DATASET == 2:
    # Copy the dataset to the local environment
    !cp "/content/drive/MyDrive/Kaggle dataset/Reddit Top 25 DJIA/KAG_REDDIT_WRLD_DJIA_DF_corrected.csv" "KAG_REDDIT_WRLD_DJIA_DF.csv"
    !cp "/content/drive/MyDrive/Economist/ECO_BSN_DF.csv" "ECO_BSN_DF.csv"
    !cp "/content/drive/MyDrive/Economist/ECO_FNC_DF.csv" "ECO_FNC_DF.csv"
    !cp "/content/drive/MyDrive/Economist/ECO_US_DF.csv" "ECO_US_DF.csv"


    # Check the copy is succesfull -> good if no assertation error
    read = !ls
    assert read[0].find("ECO_FNC_DF.csv") != -1
    assert read[0].find("KAG_REDDIT_WRLD_DJIA_DF.csv") != -1    
    assert read[1].find("ECO_BSN_DF.csv") != -1
    assert read[1].find("ECO_US_DF.csv") != -1

    # Load the datasets 
    df_reddit = pd.read_csv('KAG_REDDIT_WRLD_DJIA_DF.csv', index_col = "Date")
    df_bsn = pd.read_csv('ECO_BSN_DF.csv', index_col = "date")
    df_fnc = pd.read_csv('ECO_FNC_DF.csv', index_col = "date")
    df_us = pd.read_csv('ECO_US_DF.csv', index_col = "date")

    # Load the stock data
    df_stock = web.DataReader("DJIA", data_source="yahoo", start="2008-08-08", 
                              end="2016-07-01")

Az adathalmazok megvizsgálása az elemein keresztül.

In [133]:
if DATASET == 2:
    # Show the dataframe
    print("Reddit")
    print(df_reddit.head())
    print("\n\nBSN ECO")
    print(df_bsn.head())
    print("\n\nFNC ECO")
    print(df_fnc.head())
    print("\n\nUS ECO")
    print(df_us.head())

Reddit
            Label  ...                                              Top25
Date               ...                                                   
2008-08-08      0  ...           b"No Help for Mexico's Kidnapping Surge"
2008-08-11      1  ...  b"So this is what it's come to: trading sex fo...
2008-08-12      0  ...  b"BBC NEWS | Asia-Pacific | Extinction 'by man...
2008-08-13      0  ...  b'2006: Nobel laureate Aleksander Solzhenitsyn...
2008-08-14      1  ...  b'Philippines : Peace Advocate say Muslims nee...

[5 rows x 26 columns]


BSN ECO
                                                        title
date                                                         
2021/02/13  Unions take on Amazon and Alphabet. Big tech w...
2021/02/13      Narendra Modi promises to privatise Air India
2021/02/13                             Diary of a plague year
2021/02/13  For Deutschland AG, Brexit goes from bad to wurst
2021/02/10  The cult of an Elon Musk or a Jack Ma has its ...


FNC EC

Azon elemek megkeresése az ECO adathalmazból ami beleesik a vizsgált időintervallumba.

In [134]:
if DATASET == 2:
    df_bsn_inspect = df_bsn[df_bsn.index < '2016/07/02']
    df_bsn_inspect = df_bsn_inspect[df_bsn_inspect.index > '2008/08/07']
    df_bsn_inspect = df_bsn_inspect.drop_duplicates()

    df_fnc_inspect = df_fnc[df_fnc.index < '2016/07/02']
    df_fnc_inspect = df_fnc_inspect[df_fnc_inspect.index > '2008/08/07']
    df_fnc_inspect = df_fnc_inspect.drop_duplicates()

    df_us_inspect = df_us[df_us.index < '2016/07/02']
    df_us_inspect = df_us_inspect[df_us_inspect.index > '2008/08/07']
    df_us_inspect = df_us_inspect.drop_duplicates()

    print("BSN ECO")
    print(df_bsn_inspect.head(2))
    print("...")
    print(df_bsn_inspect.tail(2))
    print(df_bsn_inspect.shape)

    print("\n\nFNC ECO")
    print(df_fnc_inspect.head(2))
    print("...")
    print(df_fnc_inspect.tail(2))
    print(df_fnc_inspect.shape)

    print("\n\nUS ECO")
    print(df_us_inspect.head(2))
    print("...")
    print(df_us_inspect.tail(2))
    print(df_us_inspect.shape)

    print("\n\nSummary length:\t\t" + str(len(df_bsn_inspect) + len(df_fnc_inspect) + len(df_us_inspect)))

BSN ECO
                         title
date                          
2016/06/30  Squeezing the tube
2016/06/30  From clout to rout
...
                                    title
date                                     
2008/08/14                 Going for goal
2008/08/12  Come fly the fee-filled skies
(3014, 1)


FNC ECO
                              title
date                               
2016/06/30  Prophets and profiteers
2016/06/30   The consensus crumbles
...
                              title
date                               
2008/08/14           Home economics
2008/08/10  The bear and the donkey
(3269, 1)


US ECO
                          title
date                           
2016/06/30     Exodus postponed
2016/06/23  Gloria in expansion
...
                                title
date                                 
2008/08/14      The next Billy Graham
2008/08/09  Home thoughts from abroad
(3066, 1)


Summary length:		9349


Az Economist oldalról származó adathalmazok összefűzése.

In [135]:
if DATASET == 2:
    df_eco_all = pd.concat([df_bsn_inspect, df_fnc_inspect, df_us_inspect])

    df_eco_all = df_eco_all.drop_duplicates()

    df_eco_all.sort_index(ascending=True, inplace=True)

    print("ECO MERGED")
    print(df_eco_all.head(2))
    print("...")
    print(df_eco_all.tail(2))
    print(df_eco_all.shape)

ECO MERGED
                                title
date                                 
2008/08/09  Home thoughts from abroad
2008/08/10    The bear and the donkey
...
                         title
date                          
2016/06/30    Exodus postponed
2016/06/30  Squeezing the tube
(9183, 1)


Egy naphoz tartozó azonos hírek vizsgálata.

In [136]:
if DATASET == 2:
    # Groupby by date
    dates = df_eco_all.groupby("date")

    # Summary statistic
    print("Max:")
    print(dates.describe().max())
    print("\n\nMin:")
    print(dates.describe().min())

Max:
title  count             67
       unique            67
       top       iRational?
       freq               1
dtype: object


Min:
title  count                       1
       unique                      1
       top       "Rewarding failure”
       freq                        1
dtype: object


In [137]:
if DATASET == 2:
    dates_count = [] # for count
    dates_dates = [] # for indexing
    df_dates = dates.describe()

    for row in range(len(df_dates)):
        dates_count.append(len(dates.get_group(df_dates.index[row])))
        dates_dates.append(dates.get_group(df_dates.index[row]).index[0])

    df_group_dates = pd.DataFrame()
    df_group_dates["date"] = dates_dates
    df_group_dates["count"] = dates_count
    df_group_dates.set_index("date", inplace=True)
    df_group_dates.sort_index(ascending=True, inplace=True)

    print(df_group_dates.head())

            count
date             
2008/08/09      1
2008/08/10      1
2008/08/12      1
2008/08/14     19
2008/08/15      1


In [138]:
if DATASET == 2:
    # Groupby by date
    counts = df_group_dates.groupby("count")

    keys = list(counts.groups.keys())

    sum = 0

    for key in keys:
      sum = sum + key * len(counts.get_group(key))
      print("Count: " + str(key) + "\t\t" + str(len(counts.get_group(key))))

    print("\n\nSummary:\t\t" + str(sum))  

Count: 1		319
Count: 2		125
Count: 3		45
Count: 4		15
Count: 5		8
Count: 6		10
Count: 7		14
Count: 8		12
Count: 9		12
Count: 10		12
Count: 11		18
Count: 12		17
Count: 13		12
Count: 14		13
Count: 15		16
Count: 16		13
Count: 17		23
Count: 18		28
Count: 19		24
Count: 20		42
Count: 21		40
Count: 22		46
Count: 23		38
Count: 24		36
Count: 25		19
Count: 26		7
Count: 28		2
Count: 32		1
Count: 34		1
Count: 36		1
Count: 46		1
Count: 67		1


Summary:		9183


Az összefűzött hírekhez a címkék generálása a részvény árfolyama alapján.

In [139]:
if DATASET == 2:
    days = []
    stock_days = []
    wrong_days = []

    # Create dates and remove duplicates
    for day in range(len(df_eco_all.index)):
        if day == 0:
            days.append(str(df_eco_all.index[day]))
        elif df_eco_all.index[day] != days[len(days) - 1]:
            days.append(str(df_eco_all.index[day]))

    # Drop not needed days
    for day in range(len(df_stock.index)):
        stock_days.append(str(df_stock.index[day])[0:10].replace("-","/"))

    # Remove not relevant date
    good_days = []
    for day in days:
        try:
            if stock_days.index(day):
                good_days.append(str(day))
        except:
            wrong_days.append(str(day))

    print("All days:\t\t" + str(len(days)))
    print("Good days:\t\t" + str(len(good_days)))
    print("Wrong days:\t\t" + str(len(wrong_days)))

All days:		971
Good days:		666
Wrong days:		305


In [142]:
if DATASET == 2:
    label_eco = []
    date_label_eco =[]
    title_label_eco = []

    for day in range(len(good_days)):
        if day == 0:
            title_label_eco.append(df_eco_all["title"][good_days[day]])
            label_eco.append(0)
            date_label_eco.append(good_days[day])      
        # label should be 1 -> rise
        elif int(df_stock["Adj Close"][stock_days.index(good_days[day])]) >= int(df_stock["Adj Close"][stock_days.index(good_days[day]) - 1]):   
            if isinstance(df_eco_all["title"][good_days[day]], str) is False:
                for row in df_eco_all["title"][good_days[day]]:
                    title_label_eco.append(row)
                    label_eco.append(1)
                    date_label_eco.append(good_days[day])
            else:
                    title_label_eco.append(df_eco_all["title"][good_days[day]])
                    label_eco.append(1)
                    date_label_eco.append(good_days[day])

        # label should be 0 -> fall
        elif int(df_stock["Adj Close"][stock_days.index(good_days[day])]) < int(df_stock["Adj Close"][stock_days.index(good_days[day]) - 1]):   
            if isinstance(df_eco_all["title"][good_days[day]], str) is False:
                for row in df_eco_all["title"][good_days[day]]:
                    title_label_eco.append(row)
                    label_eco.append(0)
                    date_label_eco.append(good_days[day])
            else:
                    title_label_eco.append(df_eco_all["title"][good_days[day]])
                    label_eco.append(0)
                    date_label_eco.append(good_days[day])

    print("News with labels length:\t\t" + str(len(label_eco)))

News with labels length:		5069


A címkékkel rendelkező, használható adatokból egy új adathalmaz létrehozása.

In [143]:
if DATASET == 2:
    df_eco = pd.DataFrame()
    df_eco["date"] = date_label_eco
    df_eco["label"] = label_eco
    df_eco["title"] = title_label_eco
    df_eco.set_index("date", inplace=True)
    df_eco.sort_index(ascending=True, inplace=True)
    print(df_eco.head())
    print(len(df_eco))

    # drop duplicates
    df_eco.drop_duplicates(subset="title", inplace=True)
    print("\n\n ----- Drop duplicate title -----\n")
    print(df_eco.head())
    print(len(df_eco))    

            label                          title
date                                            
2008/08/12      0  Come fly the fee-filled skies
2008/08/14      1              Kicked in the ARS
2008/08/14      1          The next Billy Graham
2008/08/14      1     Another inconvenient truth
2008/08/14      1                 Phantom menace
5069


 ----- Drop duplicate title -----

            label                          title
date                                            
2008/08/12      0  Come fly the fee-filled skies
2008/08/14      1              Kicked in the ARS
2008/08/14      1          The next Billy Graham
2008/08/14      1     Another inconvenient truth
2008/08/14      1                 Phantom menace
5069


### Adathalmazok előkészítése

Az adathalmaz megtisztítása.

In [144]:
if DATASET == 2:
    # Removing punctuations
    temp_news = []
    news_sum = df_eco["title"]

    for line in news_sum:
      temp_attach = ""
      try:
          for word in line:
            temp = " "
            if word not in string.punctuation:
              temp = word
            temp_attach = temp_attach + "".join(temp)
      except:
          temp = " "
          temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    news_sum = temp_news
    temp_news = []

    # Remove numbers
    for line in news_sum:
      temp_attach = ""
      for word in line:
        temp = " "
        if not word.isdigit():
          temp = word
        temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    # Remove space
    for line in range(len(temp_news)):    
      temp_news[line] = " ".join(temp_news[line].split())

    # Converting headlines to lower case
    for line in range(len(temp_news)): 
        temp_news[line] = temp_news[line].lower()

    # Update the data frame
    df_eco["title"] = temp_news

    # Load the stop words
    stop_words = set(stopwords.words('english'))

    filtered_sentence = []
    news_sum = df_eco["title"]

    # Remove stop words
    for line in news_sum:
      word_tokens = word_tokenize(line)
      temp_attach = ""
      for word in word_tokens:
        temp = " "
        if not word in stop_words:
          temp = temp + word
        temp_attach = temp_attach + "".join(temp)
      filtered_sentence.append(temp_attach)

    # Remove space
    for line in range(len(filtered_sentence)):    
      filtered_sentence[line] = " ".join(filtered_sentence[line].split())

    # Update the data frame
    df_eco["title"] = filtered_sentence

    # Reset the index
    df_eco.reset_index(inplace=True)

    news_sum = df_eco["title"]
    null_indexes = []
    index = 0

    for line in news_sum:
      if line is "":
        null_indexes.append(index)
      index = index + 1

    print(null_indexes)

    for row in range(len(null_indexes)):
      df_eco = df_eco.drop(df_eco.index[null_indexes[row] - row])

    news_sum = df_eco["title"]
    null_indexes = []
    index = 0

    for line in news_sum:
      if line is "":
        null_indexes.append(index)
      index = index + 1
      
    assert len(null_indexes) is 0

[79, 188, 722, 1417, 1983, 2716, 3105, 3214, 3855, 4094, 4335, 4950]


Az adathalmaz szétbontása tanító és tesztelő adathalmazra.

In [145]:
if DATASET == 2:
    # Drop the dates
    df_eco_label_title = pd.DataFrame()
    df_eco_label_title["label"] = df_eco["label"]
    df_eco_label_title["title"] = df_eco["title"]
    print("New dataset without the dates")
    print(df_eco_label_title.head())
    print(len(df_eco_label_title))

    # Do the shuffle
    for i in range(SHUFFLE_CYCLE):
      df_eco_label_title = shuffle(df_eco_label_title, random_state = RANDOM_SEED)

    # Reset the index
    df_eco_label_title.reset_index(inplace=True, drop=True)

    # Show the data frame
    print("\n\nAfter shuffle")
    print(df_eco_label_title.head())    

    # Split the dataset
    INPUT_SIZE = len(df_eco_label_title)
    TRAIN_SIZE = int(TRAIN_SPLIT * INPUT_SIZE) 
    TEST_SIZE = int(TEST_SPLIT * INPUT_SIZE)

    train = df_eco_label_title[:TRAIN_SIZE] 
    test = df_eco_label_title[TRAIN_SIZE:]

    # Print out the length
    print("\n\nAfter split")
    print("Train data set length: " + str(len(train)))
    print("Test data set length: " + str(len(test)))
    print("Split summa: " + str(len(train) + len(test)))
    print("Dataset summa before split: " + str(len(df_eco_label_title)))

    # check
    split_sum = len(train) + len(test)
    sum = len(df_eco_label_title)
    assert split_sum == sum

New dataset without the dates
   label                       title
0      0   come fly fee filled skies
1      1                  kicked ars
2      1           next billy graham
3      1  another inconvenient truth
4      1              phantom menace
5057


After shuffle
   label                         title
0      0                   close power
1      0                 russian bears
2      1                    money shot
3      1                 stuck neutral
4      1  one listens jürgen grossmann


After split
Train data set length: 4298
Test data set length: 759
Split summa: 5057
Dataset summa before split: 5057


### n-gram modell

Automatikus tanítás és eredmények megjelenítése a legmagasabb korrelációs tényezőjű szavakkal együtt.

In [146]:
if DATASET == 2:
    model_type = []
    result = []
    model_type_values = []
    train_headlines = []
    test_headlines = []

    # Create model type values
    for value in range(1,7):
        model_type_values.append(value)

    for row in range(0, len(train.index)):
        train_headlines.append(train.iloc[row, 1])

    for row in range(0,len(test.index)):
        test_headlines.append(test.iloc[row, 1])


    for MODEL_TYPE in model_type_values:

        for n in range(1,MODEL_TYPE+1):
            print("--------------------------------------------\n\nStart of the " 
                  + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

            _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
            _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

            print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

            _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
            _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["label"])

            _gram_test_ = _gram_vectorizer_.transform(test_headlines)
            _gram_predictions_ = _gram_model_.predict(_gram_test_)

            print (accuracy_score(test["label"], _gram_predictions_))

            model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
            result.append(accuracy_score(test["label"], _gram_predictions_))

    best_model_gram = 0

    print("\n\n")
    for model in range(len(model_type)):
        print(str(model_type[model]) + ":\t\t\t\t\t" + str(result[model]))

        if result[model] > best_model_gram:
            best_model_gram = result[model]
            best_model_gram_index = model

    print("--------------------------------------------\nBest model:\n" 
          + str(model_type[best_model_gram_index]) + "\t\t\t\t\t" + 
          str(result[best_model_gram_index]))

--------------------------------------------

Start of the 1,1 gram model

The shape is: (4298, 4955)

0.5362318840579711
--------------------------------------------

Start of the 1,2 gram model

The shape is: (4298, 10406)

0.5362318840579711
--------------------------------------------

Start of the 2,2 gram model

The shape is: (4298, 5451)

0.5349143610013175
--------------------------------------------

Start of the 1,3 gram model

The shape is: (4298, 12212)

0.544137022397892
--------------------------------------------

Start of the 2,3 gram model

The shape is: (4298, 7257)

0.5349143610013175
--------------------------------------------

Start of the 3,3 gram model

The shape is: (4298, 1806)

0.538866930171278
--------------------------------------------

Start of the 1,4 gram model

The shape is: (4298, 12793)

0.544137022397892
--------------------------------------------

Start of the 2,4 gram model

The shape is: (4298, 7838)

0.5349143610013175
------------------------

In [148]:
if DATASET == 2:
    MODEL_TYPE = str(model_type[best_model_gram_index])

    # show the first
    print(train_headlines[0])

    _gram_vectorizer_ = CountVectorizer(ngram_range=(int(MODEL_TYPE[0]),int(MODEL_TYPE[2])))
    _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

    print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

    _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["label"])

    _gram_test_ = _gram_vectorizer_.transform(test_headlines)
    _gram_predictions_ = _gram_model_.predict(_gram_test_)

    print (accuracy_score(test["label"], _gram_predictions_))

    model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
    result.append(accuracy_score(test["label"], _gram_predictions_))

close power
The shape is: (4298, 12212)

0.544137022397892


In [149]:
if DATASET == 2:
    _gram_words_best_ = _gram_vectorizer_.get_feature_names()
    _gram_coeffs_best_ = _gram_model_.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : _gram_words_best_, 
                            'Coefficient' : _gram_coeffs_best_})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])

    print(coeffdf.head(10))

             Word  Coefficient
2751         debt     1.097224
10891    thinking     1.040934
7128       nation     0.967841
8955         ride     0.953603
10181       stand     0.936049
9643         show     0.909019
6433         make     0.902951
222      american     0.885956
11422  university     0.884117
6106       lights     0.875793


In [150]:
if DATASET == 2:
    print(coeffdf.tail(10))

              Word  Coefficient
1327         brink    -0.825932
7503        office    -0.827585
5744          kind    -0.835123
170    alternative    -0.836957
11393      unhappy    -0.837685
9278         saved    -0.854158
5639         judge    -0.882916
11149      trickle    -0.895313
9005          risk    -0.905173
6154        liquid    -1.155973


## **KAG_BENZ_ANALYST_DF, KAG_BENZ_PARTNER_DF 2008-2016**

Megvizsgálom a reddit-es világhírekkel megegyező intervallumon ezeket az összevont adathalmazokat, majd egyesítve és kombinálva a kettőt megvizsgálom, hogy javítja-e a pontosságot.

Ezen adathalmazok forrása:

*   https://www.kaggle.com/miguelaenlle/massive-stock-news-analysis-db-for-nlpbacktests 

#### Adathalmazok betöltése

In [ ]:
if DATASET == 3:
    # Copy the dataset to the local environment
    !cp "/content/drive/MyDrive/Kaggle dataset/Reddit Top 25 DJIA/KAG_REDDIT_WRLD_DJIA_DF_corrected.csv" "KAG_REDDIT_WRLD_DJIA_DF.csv"
    !cp "/content/drive/MyDrive/Kaggle dataset/Benzinga news with ticker/KAG_BENZ_ANALYST_DF_1.csv" "KAG_BENZ_ANALYST_DF_1.csv"
    !cp "/content/drive/MyDrive/Kaggle dataset/Benzinga news with ticker/KAG_BENZ_ANALYST_DF_2.csv" "KAG_BENZ_ANALYST_DF_2.csv"
    !cp "/content/drive/MyDrive/Kaggle dataset/Benzinga news with ticker/KAG_BENZ_PARTNER_DF_1.csv" "KAG_BENZ_PARTNER_DF_1.csv"
    !cp "/content/drive/MyDrive/Kaggle dataset/Benzinga news with ticker/KAG_BENZ_PARTNER_DF_2.csv" "KAG_BENZ_PARTNER_DF_2.csv"


    # Check the copy is succesfull -> good if no assertation error
    read = !ls
    assert read[1].find("KAG_BENZ_ANALYST_DF_1.csv") != -1
    assert read[2].find("KAG_REDDIT_WRLD_DJIA_DF.csv") != -1    
    assert read[2].find("KAG_BENZ_ANALYST_DF_2.csv") != -1
    assert read[0].find("KAG_BENZ_PARTNER_DF_1.csv") != -1    
    assert read[1].find("KAG_BENZ_PARTNER_DF_2.csv") != -1

    # Load the datasets 
    df_reddit = pd.read_csv('KAG_REDDIT_WRLD_DJIA_DF.csv', index_col = "Date")
    df_benz_1 = pd.read_csv('KAG_BENZ_ANALYST_DF_1.csv', index_col = "date")
    df_benz_2 = pd.read_csv('KAG_BENZ_ANALYST_DF_2.csv', index_col = "date")    
    df_partner_1 = pd.read_csv('KAG_BENZ_PARTNER_DF_1.csv', index_col = "date")
    df_partner_2 = pd.read_csv('KAG_BENZ_PARTNER_DF_2.csv', index_col = "date")

    # Load the stock data
    df_stock = web.DataReader("DJIA", data_source="yahoo", start="2008-08-08", 
                              end="2016-07-01")

A szétbontott adathalmazok összefűzése, majd azok megjelenítése.

In [ ]:
if DATASET == 3:
    # Merge them
    df_benz = pd.concat([df_benz_1, df_benz_2])
    df_partner = pd.concat([df_partner_1, df_partner_2])

    # Show the dataframe
    print("BENZ")
    print(df_benz.head())
    print("...")
    print(df_benz.tail())
    print(len(df_benz))
    print("\n\nPARTNER")
    print(df_partner.head())
    print("...")
    print(df_partner.tail())
    print(len(df_partner))

A vizsgált időtartamba eső adatok kiszűrése.

In [ ]:
if DATASET == 3:
    df_benz_inspect = df_benz[df_benz.index < '2016/07/02']
    df_benz_inspect = df_benz_inspect[df_benz_inspect.index > '2008/08/07']
    df_benz_inspect = df_benz_inspect.drop_duplicates()

    df_partner_inspect = df_partner[df_partner.index < '2016/07/02']
    df_partner_inspect = df_partner_inspect[df_partner_inspect.index > '2008/08/07']
    df_partner_inspect = df_partner_inspect.drop_duplicates()

    print("BENZ")
    print(df_benz_inspect.head(2))
    print("...")
    print(df_benz_inspect.tail())
    print(df_benz_inspect.shape)

    print("\n\nPARTNER")
    print(df_partner_inspect.head())
    print("...")
    print(df_partner_inspect.tail())
    print(df_partner_inspect.shape)

    df_benz = pd.concat([df_benz_inspect, df_partner_inspect])

    print("\n\nSummary length:\t\t" + str(len(df_benz_inspect) + len(df_partner_inspect)))

Az adathalmazban található adatok kiszűrése a Dow Jonews Industrial Average alapján:


*   Procter & Gamble, PG, 1932-05-26
*   3M Company, MMM, 1976-08-09
*   IBM, IBM, 1979-06-29
*   Merck & Co., MRK, 1979-06-29
*   American Express, AXP, 1982-08-30
*   McDonald's, MCD, 1985-10-30
*   Boeing, BA, 1987-03-12
*   The Coca-Cola Company, KO, 1987-03-12
*   Caterpillar Inc., CAT, 1991-05-06
*   JPMorgan Chase, JPM, 1991-05-06
*   The Walt Disney Company, DIS, 1991-05-06
*   Johnson & Johnson, JNJ, 1997-03-17
*   Walmart, WMT, 1997-03-17
*   The Home Depot, HD, 1999-11-01
*   Intel, INTC, 1999-11-01
*   Microsoft, MSFT, 1999-11-01
*   Verizon, VZ, 2004-04-08
*   Chevron Corporation, CVX, 2008-02-19
*   Cisco Systems, CSCO, 2009-06-08
*   The Travelers Companies, 	TRV, 2009-06-08
*   UnitedHealth Group, UNH, 	2012-09-24
*   Goldman Sachs, GS, 	2013-09-20	
*   Nike, NKE, 2013-09-20
*   Visa Inc., 	V, 2013-09-20	



In [ ]:
if DATASET == 3:
    tickers = []
    unique_count = []

    # The stock tickers which is needed
    stock_ticker = ["PG", "MMM", "IBM", "MRK", "AXP", "MCD", "BA", "KO", "CAT", "JPM",
                    "DIS", "JNJ", "WMT", "HD", "INTC", "MSFT", "VZ", "CVX", "CSCO",
                    "TRV", "UNH", "GS", "NKE", "V"]

    stocks_benz = df_benz.groupby("stock")
    stock_benz_df = stocks_benz.describe()

    for stock in range(len(stock_benz_df.index)):
        tickers.append(stock_benz_df.index[stock])

    for stock in stock_ticker:
        try:
            unique_count.append(stock_benz_df.iloc[tickers.index(stock), :][1]) #unique
        except:
            unique_count.append(0)
            print(str(stock) + "\tis not in list")

    print("\n\t---------------------------------------\n")

    sum_count = 0

    for stock in range(len(stock_ticker)):
        print(str(stock_ticker[stock]) + "\t\t\t" + str(unique_count[stock]))
        sum_count = sum_count + unique_count[stock]

    print("\n\t---------------------------------------\n")
    print("Summary of news with the tickers:\t" + str(sum_count))

In [ ]:
if DATASET == 3:
    df_benz_filtered = pd.DataFrame()

    for stock in stock_ticker:
        df_temp = df_benz[(df_benz["stock"]) == stock].drop_duplicates()
        df_benz_filtered = pd.concat([df_benz_filtered, df_temp])

    df_benz_filtered.sort_index(ascending=True, inplace=True)

    df_benz = df_benz_filtered
    df_benz.drop("stock", axis = 1, inplace = True)
    df_benz.drop_duplicates(inplace=True)

    print("BENZ")
    print(df_benz.head(2))
    print("...")
    print(df_benz.tail(2))
    print(df_benz.shape)

### A szöveg előkészítése

A szöveg előfeldolgozása következik, mint az írásjelek eltűvolítása, a számok eltávolítása, felesleges szóközöktől való megtisztítás, minden szó kisbetűs szóra cserélése.

In [ ]:
if DATASET == 3:
    # Removing punctuations
    temp_news = []
    news_sum = df_benz["headline"]

    for line in news_sum:
      temp_attach = ""
      for word in line:
        temp = " "
        if word not in string.punctuation:
          temp = word
        temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    news_sum = temp_news
    temp_news = []

    # Remove numbers
    for line in news_sum:
      temp_attach = ""
      for word in line:
        temp = " "
        if not word.isdigit():
          temp = word
        temp_attach = temp_attach + "".join(temp)
      temp_news.append(temp_attach)

    # Remove space
    for line in range(len(temp_news)):    
      temp_news[line] = " ".join(temp_news[line].split())

    # Converting headlines to lower case
    for line in range(len(temp_news)): 
        temp_news[line] = temp_news[line].lower()

    # Update the data frame
    df_benz["headline"] = temp_news

    # Load the stop words
    stop_words = set(stopwords.words('english'))

    filtered_sentence = []
    news_sum = df_benz["headline"]

    # Remove stop words
    for line in news_sum:
      word_tokens = word_tokenize(line)
      temp_attach = ""
      for word in word_tokens:
        temp = " "
        if not word in stop_words:
          temp = temp + word
        temp_attach = temp_attach + "".join(temp)
      filtered_sentence.append(temp_attach)

    # Remove space
    for line in range(len(filtered_sentence)):    
      filtered_sentence[line] = " ".join(filtered_sentence[line].split())

    # Update the data frame
    df_benz["headline"] = filtered_sentence

    print("BENZ")
    print(df_benz.head(2))
    print("...")
    print(df_benz.tail(2))
    print(df_benz.shape)

### Címke létrehozása, adathalmaz felbontása

A következőkben a címkék generálása és az adathalmaz felbontása történik tanító és validáló halmazra.

In [ ]:
if DATASET == 3:
    days = []
    stock_days = []
    wrong_days = []

    # Create dates and remove duplicates
    for day in range(len(df_benz.index)):
        temp = str(df_benz.index[day])[0:10].replace("-","/")
        if day == 0:
            days.append(temp)
        elif df_benz.index[day] != df_benz.index[day - 1]:
            days.append(temp)

    # Update the dataframe date column
    df_benz.reset_index(inplace=True)
    temp_days = df_benz["date"]
    days_to_update = []
    for date in range(len(temp_days)):
        temp = str(temp_days[date])[0:10].replace("-","/")
        days_to_update.append(temp)

    df_benz["date"] = days_to_update
    df_benz.set_index("date", inplace=True, drop=True)    

    # Drop not needed days
    for day in range(len(df_stock.index)):
        stock_days.append(str(df_stock.index[day])[0:10].replace("-","/"))

    # Remove not relevant date
    good_days = []
    for day in days:
        try:
            if stock_days.index(day):
                good_days.append(str(day))
        except:
            wrong_days.append(str(day))

    print("All days:\t\t\t\t" + str(len(days)))
    print("Good days:\t\t\t\t" + str(len(good_days)))
    print("Wrong days:\t\t\t\t" + str(len(wrong_days)))

    label_benz = []
    date_label_benz =[]
    title_label_benz = []

    for day in range(len(good_days)):
        if day == 0:
            title_label_benz.append(df_benz["headline"][good_days[day]])
            label_benz.append(0)
            date_label_benz.append(good_days[day])      
        # label should be 1 -> rise
        elif int(df_stock["Adj Close"][stock_days.index(good_days[day])]) >= int(df_stock["Adj Close"][stock_days.index(good_days[day]) - 1]):   
            if isinstance(df_benz["headline"][good_days[day]], str) is False:
                for row in df_benz["headline"][good_days[day]]:
                    title_label_benz.append(row)
                    label_benz.append(1)
                    date_label_benz.append(good_days[day])
            else:
                    title_label_benz.append(df_benz["headline"][good_days[day]])
                    label_benz.append(1)
                    date_label_benz.append(good_days[day])

        # label should be 0 -> fall
        elif int(df_stock["Adj Close"][stock_days.index(good_days[day])]) < int(df_stock["Adj Close"][stock_days.index(good_days[day]) - 1]):   
            if isinstance(df_benz["headline"][good_days[day]], str) is False:
                for row in df_benz["headline"][good_days[day]]:
                    title_label_benz.append(row)
                    label_benz.append(0)
                    date_label_benz.append(good_days[day])
            else:
                    title_label_benz.append(df_benz["headline"][good_days[day]])
                    label_benz.append(0)
                    date_label_benz.append(good_days[day])

    print("News with labels length:\t\t" + str(len(label_benz)))

In [ ]:
if DATASET == 3:
    df_benz = pd.DataFrame()
    df_benz["date"] = date_label_benz
    df_benz["label"] = label_benz
    df_benz["title"] = title_label_benz
    df_benz.set_index("date", inplace=True)
    df_benz.sort_index(ascending=True, inplace=True)
    print(df_benz.head())
    print("...")
    print(df_benz.tail())
    print(df_benz.shape)

    # drop duplicates
    df_benz.drop_duplicates(subset="title", inplace=True)
    print("\n\n ----- Drop duplicate title -----\n")
    print(df_benz.head())
    print("...")
    print(df_benz.tail())
    print(df_benz.shape) 

Az adathalmaz felbontása.

In [ ]:
if DATASET == 3:
    # Drop the dates
    df_benz_label_title = pd.DataFrame()
    df_benz_label_title["label"] = df_benz["label"]
    df_benz_label_title["title"] = df_benz["title"]
    # Reset the index
    df_benz_label_title.reset_index(inplace=True, drop=True)
    print("New dataset without the dates")
    print(df_benz_label_title.head())
    print(len(df_benz_label_title))

    # Do the shuffle
    for i in range(SHUFFLE_CYCLE):
      df_benz_label_title = shuffle(df_benz_label_title, random_state = RANDOM_SEED)

    # Reset the index
    df_benz_label_title.reset_index(inplace=True, drop=True)

    # Show the data frame
    print("\n\nAfter shuffle")
    print(df_benz_label_title.head())    

    # Split the dataset
    INPUT_SIZE = len(df_benz_label_title)
    TRAIN_SIZE = int(TRAIN_SPLIT * INPUT_SIZE) 
    TEST_SIZE = int(TEST_SPLIT * INPUT_SIZE)

    train = df_benz_label_title[:TRAIN_SIZE] 
    test = df_benz_label_title[TRAIN_SIZE:]

    # Print out the length
    print("\n\nAfter split")
    print("Train data set length: " + str(len(train)))
    print("Test data set length: " + str(len(test)))
    print("Split summa: " + str(len(train) + len(test)))
    print("Dataset summa before split: " + str(len(df_benz_label_title)))

    # check
    split_sum = len(train) + len(test)
    sum = len(df_benz_label_title)
    assert split_sum == sum

### n-gram modell

Automatikus tanítás és eredmények megjelenítése a legmagasabb korrelációs tényezőjű szavakkal együtt.

In [ ]:
if DATASET == 3:
    model_type = []
    result = []
    model_type_values = []
    train_headlines = []
    test_headlines = []

    # Create model type values
    for value in range(1,7):
        model_type_values.append(value)

    for row in range(0, len(train.index)):
        train_headlines.append(train.iloc[row, 1])

    for row in range(0,len(test.index)):
        test_headlines.append(test.iloc[row, 1])


    for MODEL_TYPE in model_type_values:

        for n in range(1,MODEL_TYPE+1):
            print("--------------------------------------------\n\nStart of the " 
                  + str(n) + "," + str(MODEL_TYPE) + " gram model\n")

            _gram_vectorizer_ = CountVectorizer(ngram_range=(n,MODEL_TYPE))
            _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

            print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

            _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
            _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["label"])

            _gram_test_ = _gram_vectorizer_.transform(test_headlines)
            _gram_predictions_ = _gram_model_.predict(_gram_test_)

            print (accuracy_score(test["label"], _gram_predictions_))

            model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
            result.append(accuracy_score(test["label"], _gram_predictions_))

    best_model_gram = 0

    print("\n\n")
    for model in range(len(model_type)):
        print(str(model_type[model]) + ":\t\t\t\t\t" + str(result[model]))

        if result[model] > best_model_gram:
            best_model_gram = result[model]
            best_model_gram_index = model

    print("--------------------------------------------\nBest model:\n" 
          + str(model_type[best_model_gram_index]) + "\t\t\t\t\t" + 
          str(result[best_model_gram_index]))

In [ ]:
if DATASET == 3:
    MODEL_TYPE = str(model_type[best_model_gram_index])

    # show the first
    print(train_headlines[0])

    _gram_vectorizer_ = CountVectorizer(ngram_range=(int(MODEL_TYPE[0]),int(MODEL_TYPE[2])))
    _train_vectorizer_ = _gram_vectorizer_.fit_transform(train_headlines)

    print("The shape is: " + str(_train_vectorizer_.shape) + "\n")

    _gram_model_ = LogisticRegression(random_state=RANDOM_SEED, max_iter=MAX_ITER)
    _gram_model_ = _gram_model_.fit(_train_vectorizer_, train["label"])

    _gram_test_ = _gram_vectorizer_.transform(test_headlines)
    _gram_predictions_ = _gram_model_.predict(_gram_test_)

    print (accuracy_score(test["label"], _gram_predictions_))

    model_type.append(str(n) + "," + str(MODEL_TYPE) + " n-gram")
    result.append(accuracy_score(test["label"], _gram_predictions_))

In [ ]:
if DATASET == 3:
    _gram_words_best_ = _gram_vectorizer_.get_feature_names()
    _gram_coeffs_best_ = _gram_model_.coef_.tolist()[0]

    coeffdf = pd.DataFrame({'Word' : _gram_words_best_, 
                            'Coefficient' : _gram_coeffs_best_})

    coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])

    print(coeffdf.head(10))

In [ ]:
if DATASET == 3:
    print(coeffdf.tail(10))